# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Построим модель с максимально большим значением *accuracy* - нужно довести долю правильных ответов по крайней мере до 0.75. Затем проверим *accuracy* на тестовой выборке.

## Откроем и изучим файл

In [98]:
# Загрузим все необходимые библиотеки:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [99]:
# Выведем данные:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Данные уже подготовлены к классификации: есть столбец is_ultra - каким тарифом пользовался абонент в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разделим данные на выборки

In [100]:
# Извлечем признаки и целевой признак:
features = df.drop(['is_ultra'], axis=1)  
target = df['is_ultra']

# Разделим данные на тренировочную, валидационную и тестовую выборки (3,1,1): 
features_train, features_split, target_train, target_split = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_split, target_split, test_size=0.5, random_state=12345)

# Распечатаем размеры выборок:
data = features_train, target_train, features_valid, target_valid, features_test, target_test

for i in data:
    display(i.shape)

(1928, 4)

(1928,)

(643, 4)

(643,)

(643, 4)

(643,)

## Исследуем модели

In [101]:
# Применим дерево решений для создания модели с глубиной от 1 до 5:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_1.fit(features_train, target_train) 
    result = model_1.score(features_valid, target_valid)
    if result > best_result:
        best_model = model_1
        best_result = result
        best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 Глубина дерева: 3


Наилучший результат среди моделей дерева решений показала модель с глубиной дерева 3.

In [102]:
# Применим случайный лес для создания модели:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)  
        model_2.fit(features_train, target_train) 
        result = model_2.score(features_valid, target_valid)
        if result > best_result:
            best_model = model_2
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, 
      "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 10


Наилучший результат показала модель случайного леса с количеством деревьев 40 и глубиной 10, accuracy - 0.81.

In [103]:
# Применим логистическую регрессию для создания модели:
model_3 = LogisticRegression() 
model_3.fit(features_train, target_train) 
result = model_3.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Модель логистической регрессии имеет наихудший accuracy - 0.75.

**Вывод:**
Наилучший результат среди моделей показал случайный лес с количеством деревьев 40 и глубиной 10: accuracy = 0.81.

## Проверим модель на тестовой выборке

In [104]:
# Проверим модель случайного леса на тестовой выборке:
predictions_test = model_2.predict(features_test) 

accuracy_score(target_test, predictions_test)

0.8009331259720062

Удалось достичь accuracy в 0.80.

## Проверим модель на адекватность

In [106]:
# Используем DummyClassifier для проверки модели на адекватность:
dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(features_train, target_train)

dummy_pred = dummy_clf.predict(features_test)

accuracy_score(target_test, dummy_pred)

0.6842923794712286

## Выводы

Мы исследовали различные модели и выбрали среди них модель случайного леса, которая имеет наилучший accuracy - доля правильных ответов составила 0.81. Мы проверили модель на тестовой выборке: accuracy составила 0.8. Также мы проверили модель на адекватность: accuracy - 0.68.